In [ ]:
import re
import json
import openai
OPENAI_API_KEY = ''
openai.api_key = OPENAI_API_KEY


def ndesc_generator(grammar_pattern, gp_desc, group, g_desc, group_mm, examples):
    # openai.ChatCompletion.create
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
                {'role':'system', 'content':'You serve as a professional English teacher.'}, 
                {'role':'user', 'content':f"""In grammar pattern "{grammar_pattern}", {gp_desc} 
                                            Nouns exhibiting this pattern can be categorized into semantic groups.
                                            In '{group}': {g_desc}"""}, 
                {'role':'assistant', 'content':f"""{GROUP_MM} belobg to '{group}'.
                and here are some example sentences: {examples}"""}, 
                {"role": "user",
                "content": f"""The noun group in {grammar_pattern} of '{group}' indicates [MASK]."""}
        ],
        temperature=0,
        max_tokens=1500,
    )
    # generalize, induce
    response_text = response.choices[0].message.content
    return response_text

def extract_partial_sentence(text):
    pattern = r'indicates(.*)'
    match = re.search(pattern, text)

    if match:
        partial_sentence = match.group(1).strip()
        return partial_sentence
    else:
        return None

def subject_extractor(n_desc):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"""Task: Extract the subject of sentence.
                example:
                Q: someone sympathizes with someone else.
                A: someone
                Q: {n_desc}
                A: [MASK]
                """
            },
        ],
        temperature=0,
        max_tokens=1500,
    )
    # generalize, induce
    subject = response.choices[0].message.content
    return subject



NDESC = ndesc_generator(PATTERN, GP_DESC, GROUP, G_DESC, GROUP_MM, EXAMPLE)
_ = extract_partial_sentence(NDESC)
SC = subject_extractor(_)

In [6]:
import re
import json
import openai
import nltk
from nltk.tokenize import word_tokenize
OPENAI_API_KEY = ''
openai.api_key = OPENAI_API_KEY



def sc_classifier(n_desc):
    # generate description
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": f"""You are a multi-label text classifier designed to receive descriptions of noun arguments 
                and generate the corresponding semantic categories for these noun arguments within 70 defined categories:
                "Human", "Transport", "Currency", "Illness", "Action", "Decision", "Plan", "Garment", 
                "Location", "Proposition", "Information", "Medium", "Document", "Quantity", 
                "Money Value", "Goal", "Attitude", "Emotion", "Resource", "Responsibility", 
                "Time Period", "Time Point", "Language Part", "Property", "Character Trait", 
                "Skill", "Weight", "Power", "Privilege", "Relationship", "Illustration", "Lack", 
                "Addressee", "Assembly", "Part", "Thing", "Physical Entities", "Topic", 
                "Consequence", "Change", "Strategy", "Occurrence", "Effort", "Aid", "Work", 
                "Difficulty", "Assignment", "Element", "Condition", "Sense", "Principle", "Basis", 
                "Classification", "Manifestation", "Stage", "Domain", "Industry", "Centre", 
                "Opportunity", "System", "Danger", "Verification", "Synthesis", "Aesthetics", 
                "Necessity", "Range", "Progress", "Container", "Environment", "Stimulus".\n"""
            },
            {
                "role": "user",
                "content": f"""example:
                    Q:"The noun group in ADJ against n of 'The 'steady' group' indicates the currency or entity that the first currency is being compared to.",
                    A:Currency\n
                    Q:"The noun group in N on n of 'The 'conference' group' indicates the topic or subject that the conference, seminar, symposium, etc., is focused on or discussing. For example, in the sentence "In Seoul he will attend a regional conference on trade issues," the noun group "trade issues" indicates the subject of the conference."
                    A:Topic\n
                    Q:{n_desc}, 
                    A: [MASK]\n"""
            } 
        ],
        temperature=0,
        max_tokens=1500,
    )
    # generalize, induce
    response_text = response.choices[0].message.content
    return response_text



path = f'result/_Verb_output.txt'
with open(path, 'r') as f:
    lines = f.readlines()

pattern = r'The noun group in (.*?) of'
_dict = {}
k = 0

for i, line in enumerate(lines):
    if i == 0 + 5*k:
        group = line.replace('\n', '')
    if i == 1 + 5*k:
        match = re.search(pattern, line)
        if match:
            GP = match.group(1)
            if GP not in _dict.keys():
                _dict[GP] = {}
            n_desc = line.replace('\n', '')
            _dict[GP][group] = sc_classifier(n_desc)
    
    if i == 2 + 5*k:
        #_dict[GP][group] = line.replace('\n', '')
        k += 1
